## Importando a base de dados com as notas dos cursos

In [1]:
# -*- coding: utf-8 -*-

# Entrando com os pacotes necessários 

import pandas as pd # manipula os dados em DataFrame
import psycopg2 # conecta o banco sql ao python
import pandas.io.sql as sqlio # importa os dados do banco sql em formato Dataframe

# Colocando as informações de login no servidor

host = '10.224.9.157'
dbname= 'dw_consolidado' 
username = 'postgres' 
password = 'Enap@123' 

# Realizando a coneção com o banco de dados

conn = psycopg2.connect(host = host, dbname = dbname, user = username, password = password, port='5432')

In [2]:
%%time

# Puxando o banco de dados

sql = f"select cpf, curso, nota_avaliacao from tb_fato_matricula inner join dim_aluno using (dim_aluno_id) inner join dim_origem_dado using (dim_origem_dado_id) inner join dim_acao using (dim_acao_id) inner join dim_situacao using (dim_situacao_id) left join tb_fato_indicadores using (id_fato_matricula) left join dim_avaliacao using (dim_avaliacao_id) where ((dim_avaliacao_id = 4) and (formato_acao = 'Curso')) order by data_conclusao asc;"
df_avaliacao = sqlio.read_sql_query(sql, conn) # Pegando as tabelas de interesse 

C:\Users\ferna\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Wall time: 40.8 s


## Modelo de recomendação KNN

In [3]:
df = df_avaliacao.drop_duplicates(subset=['cpf', 'curso'], keep='last')

In [4]:
import numpy as np 
import pandas as pd

# Realizando o filtro para pegar apenas resposta referente a pergunta  4

df_pivot = df.pivot_table(index = ["curso"],columns = ["cpf"],values = "nota_avaliacao").fillna(0)

In [5]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pandas as pd

curso = []
distance = []
curso_lista_indicacao = []
curso_lista = []

for query_index in range(0,len(df_pivot)):

    matriz_usuarios = csr_matrix(df_pivot.values)
    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    model_knn.fit(matriz_usuarios)
    distances, indices = model_knn.kneighbors(df_pivot.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 6)   

    for i in range(0, len(distances.flatten())):
        if i != 0:
            curso.append(df_pivot.index[indices.flatten()[i]])
            distance.append(distances.flatten()[i])    

    m=pd.Series(curso,name='curso')
    d=pd.Series(distance,name='distance')
    recommend = pd.concat([m,d], axis=1)
    recommend = recommend.sort_values('distance',ascending=False)

    # print('Recommendations for {0}:\n'.format(df_pivot.index[query_index]))
    for i in range(0,4):
        curso_lista_indicacao.append('{0}'.format(recommend["curso"].iloc[i]))

    curso_lista.append(str(df_pivot.index[query_index] + str('|') + df_pivot.index[query_index] + str('|')+ df_pivot.index[query_index] + str('|') + df_pivot.index[query_index]).split('|'))

cursos = [item for sublist in curso_lista for item in sublist]

In [6]:
dados = pd.DataFrame (cursos ,columns=['Curso'])
dados['Indicações'] = curso_lista_indicacao

# Melhorando o formato de exportação

recomendacao_1 = []

for i in list(range(1,len(dados),4)):
    recomendacao_1.append(dados['Indicações'][i])
    
recomendacao_2 = []

for i in list(range(2,len(dados),4)):
    recomendacao_2.append(dados['Indicações'][i])
    
    
recomendacao_3 = []

for i in list(range(3,len(dados),4)):
    recomendacao_3.append(dados['Indicações'][i])
    
curso = []

for i in list(range(0,len(dados),4)):
    curso.append(dados['Curso'][i])
    
    
# Criando um DataFrame para alocar os outputs

dados = pd.DataFrame (curso ,columns=['curso'])
dados['recomendacao_1'] = recomendacao_1
dados['recomendacao_2'] = recomendacao_2
dados['recomendacao_3'] = recomendacao_3

### Exportando em formato csv

dados.to_csv("dados_KNN.csv", index=False, encoding='utf-8-sig', sep = '|')

dados

,curso,recomendacao_1,recomendacao_2,recomendacao_3
0,A Liderança Pública em Tempos de Crise,Gestão Pessoal - Base da Liderança,Gestão de Comportamento nas Organizações,Liderança e Gestão De Equipes
1,A Previdência Social dos Servidores Públicos: ...,Censo Cadastral Previdenciário,Noções Básicas do Trabalho Remoto,Gestão Pessoal - Base da Liderança
2,A construção dos livros didáticos do PNLD,Censo Cadastral Previdenciário,Noções Básicas do Trabalho Remoto,Gestão Pessoal - Base da Liderança
3,A importância do PNLD,Censo Cadastral Previdenciário,Noções Básicas do Trabalho Remoto,Gestão Pessoal - Base da Liderança
4,Abertura de empresas: como redesenhar e simpli...,Contabilidade com foco na gestão da informação...,Lei de Liberdade Econômica e o Licenciamento 4.0,Accountability política
...,...,...,...,...
467,eMAG Conteudista,Sistema Eletrônico de Informações - SEI! USAR,Portal Gov.Br,O papel do DevOps na Transformação Digital dos...
468,eMAG Desenvolvedor,Sistema Eletrônico de Informações - SEI! USAR,Portal Gov.Br,O papel do DevOps na Transformação Digital dos...
469,eSocial para Órgãos Públicos - RPPS,Sistema Eletrônico de Informações - SEI! USAR,Portal Gov.Br,O papel do DevOps na Transformação Digital dos...
470,Ágil no Contexto do Serviço Público,Sistema Eletrônico de Informações - SEI! USAR,Portal Gov.Br,O papel do DevOps na Transformação Digital dos...


## Modelo de recomendação SVD

In [7]:
# Criando a matriz pivot para processamento

df_pivot = df.pivot_table(index = ["curso"],columns = ["cpf"],values = "nota_avaliacao").fillna(0)

In [8]:
from numpy.linalg import svd

matrix = df_pivot.values
u, s, vh = svd(matrix, full_matrices=False)

In [9]:
import numpy as np
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))

indicacoes = []
curso_lista = []

for query_index in range(0,len(df_pivot)):
    
    highest_similarity = -np.inf
    highest_sim_col = -1
    highest_similarity_list = []
    highest_sim_col_list = []
    

    for col in range(1,vh.shape[1]):
        similarity = cosine_similarity(vh[:,query_index], vh[:,col])
        if similarity > highest_similarity:
            highest_similarity = similarity
            highest_similarity_list.append(similarity)
            highest_sim_col = col
            highest_sim_col_list.append(col)
 
    print(f"Column %d is most similar to column {query_index}" % highest_sim_col)
    
    resultado = list(dict.fromkeys([df_pivot.iloc[:, index].idxmax() for index in highest_sim_col_list ]))[-4:]

    if len(resultado) == 1:
        resultado.append('')
        resultado.append('')
        resultado.append('')
    
    elif len(resultado) == 2:
        resultado.append('')
        resultado.append('')
        
    elif len(resultado) == 3:
        resultado.append('')
        
    indicacoes.append(resultado)
    
    curso_lista.append(str(df_pivot.index[query_index] + str('|') + df_pivot.index[query_index] + str('|')+ df_pivot.index[query_index] + str('|') + df_pivot.index[query_index]).split('|'))
    
indicacoes = [item for sublist in indicacoes for item in sublist]
cursos = [item for sublist in curso_lista for item in sublist]

<ipython-input-9-a3ef8b50df47>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))


Column 9922 is most similar to column 0
Column 1 is most similar to column 1
Column 121 is most similar to column 2
Column 5052 is most similar to column 3
Column 4 is most similar to column 4
Column 3336 is most similar to column 5
Column 6 is most similar to column 6
Column 1213 is most similar to column 7
Column 1350 is most similar to column 8
Column 9 is most similar to column 9
Column 5680 is most similar to column 10
Column 512 is most similar to column 11
Column 12 is most similar to column 12
Column 764 is most similar to column 13
Column 117 is most similar to column 14
Column 9922 is most similar to column 15
Column 1783 is most similar to column 16
Column 17 is most similar to column 17
Column 5066 is most similar to column 18
Column 19 is most similar to column 19
Column 20 is most similar to column 20
Column 317 is most similar to column 21
Column 288 is most similar to column 22
Column 348 is most similar to column 23
Column 298 is most similar to column 24
Column 461 is

In [10]:
dados = pd.DataFrame (cursos ,columns=['Curso'])
dados['Indicações'] = indicacoes


# Melhorando o formato de exportação

recomendacao_1 = []

for i in list(range(1,len(dados),4)):
    recomendacao_1.append(dados['Indicações'][i])
    
recomendacao_2 = []

for i in list(range(2,len(dados),4)):
    recomendacao_2.append(dados['Indicações'][i])
    
    
recomendacao_3 = []

for i in list(range(3,len(dados),4)):
    recomendacao_3.append(dados['Indicações'][i])
    
curso = []

for i in list(range(0,len(dados),4)):
    curso.append(dados['Curso'][i])
    
    
# Criando um DataFrame para alocar os outputs

dados = pd.DataFrame (curso ,columns=['curso'])
dados['recomendacao_1'] = recomendacao_1
dados['recomendacao_2'] = recomendacao_2
dados['recomendacao_3'] = recomendacao_3

### Exportando em formato csv

dados.to_csv("dados_SVD.csv", index=False, encoding='utf-8-sig', sep = '|')

dados

,curso,recomendacao_1,recomendacao_2,recomendacao_3
0,A Liderança Pública em Tempos de Crise,Gestão de Conflitos e Negociação,Introdução à Vigilância Sanitária,
1,A Previdência Social dos Servidores Públicos: ...,,,
2,A construção dos livros didáticos do PNLD,,,
3,A importância do PNLD,Gestão de Conflitos e Negociação,,
4,Abertura de empresas: como redesenhar e simpli...,Gestão de Conflitos e Negociação,Análise de dados como suporte à tomada de decisão,
...,...,...,...,...
467,eMAG Conteudista,Conselhos de Promoção da Igualdade Racial,Controles na Administração Pública,Gestão de Equipes em Trabalho Remoto
468,eMAG Desenvolvedor,Gestão de Conflitos e Negociação,Análise de dados como suporte à tomada de decisão,Lei nº 8112/90 e suas alterações
469,eSocial para Órgãos Públicos - RPPS,Orçamento Público,A Liderança Pública em Tempos de Crise,Segurança Química em Laboratórios de Ensino e ...
470,Ágil no Contexto do Serviço Público,Análise de dados como suporte à tomada de decisão,Segurança do paciente e Qualidade em serviços ...,Boas Práticas de Manipulação em Serviços de Al...
